In [2]:
import boto3

region = "us-east-2"
endpoint_name = "telco-churn-endpoint-v3"

sm = boto3.client("sagemaker", region_name=region)

# Delete endpoint
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
    print(f"✅ Deleted endpoint: {endpoint_name}")
except Exception as e:
    print(f"⚠️ Endpoint delete issue: {e}")

# Delete endpoint config
try:
    sm.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"✅ Deleted endpoint config: {endpoint_name}")
except Exception as e:
    print(f"⚠️ Endpoint config delete issue: {e}")


✅ Deleted endpoint: telco-churn-endpoint-v3
✅ Deleted endpoint config: telco-churn-endpoint-v3


In [3]:
import subprocess

cmd_stream = [
    "aws", "logs", "describe-log-streams",
    "--log-group-name", f"/aws/sagemaker/Endpoints/{endpoint_name}",
    "--order-by", "LastEventTime",
    "--descending", "--limit", "1",
    "--region", region
]

result = subprocess.run(cmd_stream, capture_output=True, text=True)
print("Log Stream Info:\n", result.stdout)


Log Stream Info:
 {
    "logStreams": [
        {
            "logStreamName": "AllTraffic/i-03069b578bd572554",
            "creationTime": 1756684893408,
            "firstEventTimestamp": 1756684888335,
            "lastEventTimestamp": 1756685631335,
            "lastIngestionTime": 1756685631844,
            "uploadSequenceToken": "49039859642744165107625857414532809241215742770591431137",
            "arn": "arn:aws:logs:us-east-2:484870651621:log-group:/aws/sagemaker/Endpoints/telco-churn-endpoint-v3:log-stream:AllTraffic/i-03069b578bd572554",
            "storedBytes": 0
        }
    ],
    "nextToken": "Kwfr0bfVKkYk4OrYazFRiXum2I0pXyV-62bCphD8wMCt87aMxamF9wESYg8F2jOaKaNpdpShUpjjL4q9M1S1d1w0pDBxkKuaqOA-qh0TOlbvnv3F7bEj3V83F0jhkd6N4LuKjg_FuM5ifbbDXx3-mkDJQW0W-XGrFJro1eeSKxEVGikFCNHpT-olsyHGbtxOJzPLqAAuoNjhTL66EKlSe6qSfEOth1TjiU8idqOfpfvUUcl4Z0b3Ash4Y_mp_kZ89dt-qH_R9tHIDxz3bI9fpKWiyTkAi_uZxP6TeBcsu2lFIb--kiSOqDUUR4TJUXFSgtTQ9LnEH6tQGuw8wKJcWMIE1T-WRbjgzOLT3tl8vssKu-_r2JK2tD9IkQd

In [4]:
import subprocess
import json

endpoint_name = "telco-churn-endpoint-v3"
log_group_name = f"/aws/sagemaker/Endpoints/{endpoint_name}"
output_file = "/home/studio-lab-user/sagemaker-studiolab-notebooks/telco-customer-churn-pred/notebooks/cloudwatch_logs.txt"

# Get latest log stream via describe-log-streams
cmd_stream = [
    "aws", "logs", "describe-log-streams",
    "--log-group-name", log_group_name,
    "--order-by", "LastEventTime",
    "--descending",
    "--limit", "1"
]
result = subprocess.run(cmd_stream, stdout=subprocess.PIPE)
log_streams = json.loads(result.stdout.decode("utf-8"))
log_stream_name = log_streams["logStreams"][0]["logStreamName"]

print(f"Using log stream: {log_stream_name}")

# Now fetch logs
cmd_logs = [
    "aws", "logs", "get-log-events",
    "--log-group-name", log_group_name,
    "--log-stream-name", log_stream_name,
    "--start-from-head",
    "--output", "text"
]

with open(output_file, "w") as f:
    subprocess.run(cmd_logs, stdout=f)

print(f"✅ Logs exported to {output_file}")


Using log stream: AllTraffic/i-03069b578bd572554
✅ Logs exported to /home/studio-lab-user/sagemaker-studiolab-notebooks/telco-customer-churn-pred/notebooks/cloudwatch_logs.txt


In [1]:
import boto3

# SageMaker client
sm_client = boto3.client("sagemaker")

# Replace with your current endpoint name
endpoint_name = "telco-churn-endpoint-v3"

# Check status
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint name: {response['EndpointName']}")
print(f"Status: {response['EndpointStatus']}")
print(f"Creation time: {response['CreationTime']}")
print(f"Last modified: {response['LastModifiedTime']}")
if "FailureReason" in response:
    print(f"Failure reason: {response['FailureReason']}")


Endpoint name: telco-churn-endpoint-v3
Status: Failed
Creation time: 2025-08-31 23:46:42.145000+00:00
Last modified: 2025-09-01 00:10:20.214000+00:00
Failure reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.
